In [7]:
# Dataset location (edit as needed)
import os
import pandas as pd
root_dir = '../'

In [8]:
import sys
print(sys.version)
!python --version

3.7.10 (default, Feb 26 2021, 18:47:35) 
[GCC 7.3.0]
Python 3.7.10


In [12]:
# Make dataframe csvs for train/test

out_dir = os.path.join(root_dir, 'csvs/')
os.makedirs(out_dir, exist_ok=True)
# data_dir = 'Test/Urban3D_Test'
data_dir = 'Train/Urban3D_Train'


d = os.path.join(root_dir, 'data', data_dir)
subdirs = sorted([f for f in os.listdir(d)])    
outpath = os.path.join(out_dir, data_dir.split('/')[1] + '_df.csv')
im_list, mask_list = [], []


im_files = [os.path.join( d,'RGB',  f.split('.')[0] + '.tif')
for f in sorted(os.listdir(os.path.join(d,'RGB' )))]

mask_files = [os.path.join(d, 'masks', f.split('.')[0] + '.tif')
for f in sorted(os.listdir(os.path.join(d, 'masks')))]


im_list.extend(im_files)
mask_list.extend(mask_files)


df = pd.DataFrame({'image': im_list, 'label': mask_list})
display(df.head())

df.to_csv(outpath, index=False)

print("output csv:", outpath)

,image,label
0,../data/Train/Urban3D_Train/RGB/_10_JAX_Tile_0...,../data/Train/Urban3D_Train/masks/_10_JAX_Tile...
1,../data/Train/Urban3D_Train/RGB/_10_JAX_Tile_0...,../data/Train/Urban3D_Train/masks/_10_JAX_Tile...
2,../data/Train/Urban3D_Train/RGB/_10_JAX_Tile_0...,../data/Train/Urban3D_Train/masks/_10_JAX_Tile...
3,../data/Train/Urban3D_Train/RGB/_10_JAX_Tile_0...,../data/Train/Urban3D_Train/masks/_10_JAX_Tile...
4,../data/Train/Urban3D_Train/RGB/_10_JAX_Tile_0...,../data/Train/Urban3D_Train/masks/_10_JAX_Tile...


output csv: ../csvs/Urban3D_Train_df.csv
